In [1]:
import yaml
import requests
from io import BytesIO
import os
import zipfile

In [2]:
with open("file_links.yaml", 'r') as stream:
    try:
        data = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [4]:
len(data["links"])

25

In [3]:
def download_file(url, path_os):
    response = requests.get(url, stream=True)
    if response.status_code == requests.codes.OK:
        with open(path_os, 'wb') as new_file:
                for part in response.iter_content(chunk_size=256):
                    new_file.write(part)
        print(f"Download finalizado. Arquivo salvo em: {path_os}")
    else:
        response.raise_for_status()

In [ ]:
download_file("http://200.152.38.155/CNPJ/K3241.K03200Y1.D10814.EMPRECSV.zip", "C:/Users/lays.lima/Documents/teste")

In [ ]:
response = requests.get("http://200.152.38.155/CNPJ/K3241.K03200Y1.D10814.EMPRECSV.zip", stream=True)

In [23]:
with open("C:/Users/lays.lima/Documents/DS Estudos/A3_Data_Challenge_Sept/data/", 'wb') as new_file:
        new_file.write(response.content)

PermissionError: [Errno 13] Permission denied: 'C:/Users/lays.lima/Documents/DS Estudos/A3_Data_Challenge_Sept/data/'

In [4]:
url = "http://200.152.38.155/CNPJ/K3241.K03200Y1.D10814.EMPRECSV.zip"

In [5]:
filebytes = BytesIO(requests.get(url, stream=True).content)

In [7]:
myzip = zipfile.ZipFile(filebytes)

In [8]:
myzip.extractall("data/")

In [3]:
import requests
from multiprocessing.pool import ThreadPool
 
def download_url(url):
  print("downloading: ",url)
  file_name_start_pos = url.rfind("/") + 1
  file_name = url[file_name_start_pos:]
 
  r = requests.get(url, stream=True)
  if r.status_code == requests.codes.ok:
    with open("data/" + file_name, 'wb') as f:
      for data in r:
        f.write(data)
  return url
 
with open("teste.yaml", 'r') as stream:
    try:
        data = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

urls = data["links"]
 
# Run 5 multiple threads. Each call will take the next element in urls list
results = ThreadPool(len(data["links"])).imap_unordered(download_url, urls)
for r in results:
    print(r)

downloading: downloading:  http://200.152.38.155/CNPJ/K3241.K03200Y3.D10814.EMPRECSV.zip
 downloading: downloading:  http://200.152.38.155/CNPJ/K3241.K03200Y2.D10814.EMPRECSV.zipdownloading:  http://200.152.38.155/CNPJ/K3241.K03200Y4.D10814.EMPRECSV.zip
 http://200.152.38.155/CNPJ/K3241.K03200Y6.D10814.EMPRECSV.zip
downloading: 
downloading:  http://200.152.38.155/CNPJ/K3241.K03200Y5.D10814.EMPRECSV.ziphttp://200.152.38.155/CNPJ/K3241.K03200Y7.D10814.EMPRECSV.zip
downloading:  
downloading:  http://200.152.38.155/CNPJ/K3241.K03200Y1.D10814.ESTABELE.zip downloading:  http://200.152.38.155/CNPJ/K3241.K03200Y2.D10814.ESTABELE.zip
http://200.152.38.155/CNPJ/K3241.K03200Y9.D10814.EMPRECSV.zipdownloading: http://200.152.38.155/CNPJ/K3241.K03200Y8.D10814.EMPRECSV.zip
downloading:  
downloading:  http://200.152.38.155/CNPJ/K3241.K03200Y3.D10814.ESTABELE.zip
 http://200.152.38.155/CNPJ/K3241.K03200Y5.D10814.ESTABELE.zip
downloading:  http://200.152.38.155/CNPJ/K3241.K03200Y6.D10814.ESTABELE.zip

# Retornar os links da página do governo

In [ ]:
import httplib2
from bs4 import BeautifulSoup, SoupStrainer

http = httplib2.Http()
status, response = http.request('https://www.gov.br/receitafederal/pt-br/assuntos/orientacao-tributaria/cadastros/consultas/dados-publicos-cnpj')


for link in BeautifulSoup(response, 'html.parser',
                             parseOnlyThese=SoupStrainer('a')):
    if link.has_attr('href'):
        print(link['href'])

# Usando spark

In [1]:
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
import dask.dataframe as dd

#spark = SparkSession.builder.appName('PySpark Column Ops').config("spark.sql.repl.eagerEval.enabled", True).getOrCreate()

col_empresas = ["CNPJ", "Razao_Social", "Natureza_Juridica", "Qualificacao_do_Responsavel", "Capital_Social", "Porte", "Ente_Federativo_Responsavel"]
col_estabelecimentos = ["CNPJ_Basico", "CNPJ_Ordem", "CNPJ_DV", "Identificador", "Nome_Fantasia", "Situacao_Cadastral", "Data_Situacao_Cadastral", 
                        "Motivo_Situacao_Cadastral", "Nome_Cidade_Exterior", "Pais", "Data_Inicio_Atividad", "CNAE_Fiscal_Principal", "CNAE_Fiscal_Secundaria",
                        "Tipo_Logradouro", "Logradouro", "Numero", "Complemento", "Bairro", "CEP", "UF", "Municipio", "DDD_1", "Telefone_1", "DDD_2", "Telefone_2", 
                        "DDD_Fax", "Fax", "Correio_Eletronico", "Situacao_Especial", "Data_Situacao_Especial"]

# Tratamento dos arquivos de Empresas
## Adicionando arquivos ao DataFrame

In [16]:
# Lê todos os arquivos passados como parâmetro, utilizando o dask
df_empresa = dd.read_csv('data/extract/*.EMPRECSV', delimiter=';', encoding='latin1', low_memory=False, dtype=str, names=col_empresas)

In [17]:
df_empresa.head()

,CNPJ,Razao_Social,Natureza_Juridica,Qualificacao_do_Responsavel,Capital_Social,Porte,Ente_Federativo_Responsavel
0,41273605,WALLACE DE OLIVEIRA SILVA 42992101826,2135,50,"1000,00",01,NaN
1,41273606,MARCOS CESAR DE MELO JUNIOR 02569327190,2135,50,"72000,00",01,NaN
2,41273607,LAYANE SCARLETT DE SOUZA ARAUJO 07071498199,2135,50,"1,00",01,NaN
3,41273608,FRANCISCA SAMPAIO DE OLIVEIRA,2135,50,"0,00",01,NaN
4,41273609,INGRID DIAS ALVES 03151432077,2135,50,"10000,00",01,NaN


## Tratamento da coluna Capital Social

In [18]:
df_empresa['Capital_Social'] = df_empresa['Capital_Social'].map(lambda x: x.replace(',','.'))
df_empresa['Capital_Social'] = dd.to_numeric(df_empresa['Capital_Social'], errors='coerce')

## Preenchendo os valores NA da coluna Porte com 01 = Não informado

In [19]:
df_empresa['Porte'] = df_empresa['Porte'].fillna(value='01')

In [20]:
df_empresa['Porte'].unique().compute()

0    01
1    03
2    05
Name: Porte, dtype: object

In [8]:
df_empresa.isnull().sum().compute()

CNPJ                                  0
Razao_Social                          0
Natureza_Juridica                     0
Qualificacao_do_Responsavel           0
Capital_Social                        0
Porte                             60627
Ente_Federativo_Responsavel    46846713
dtype: int64

# Manipulação dos dados de Estabelecimentos

In [2]:
df_estab = dd.read_csv('data/extract/*.ESTABELE', delimiter=';', encoding='latin1', low_memory=False, dtype=str, names=col_estabelecimentos)

In [42]:
# df_estab["CNPJ_Ordem"] = df_estab["CNPJ_Ordem"].astype(str).apply(lambda x: x.zfill(4)) # preenche os campos com zeros a esquerda até completar 4 caracteres
# df_estab["CNPJ_DV"] = df_estab["CNPJ_DV"].astype(str).apply(lambda x: x.zfill(2))

## Altera para 999 os valores nulos da coluna País

In [4]:
df_estab['Pais'] = df_estab['Pais'].fillna(value='999')

## Adicionando a coluna CNPJ concatenando as informações do CNPJ Basico, CNPJ Ordem e CNPJ DV

In [7]:
cnpjs = (df_estab["CNPJ_Basico"].astype(str).apply(lambda x: x.zfill(8), meta=('CNPJ_Basico', str)) + 
        df_estab["CNPJ_Ordem"].astype(str).apply(lambda x: x.zfill(4), meta=('CNPJ_Ordem', str)) + 
        df_estab["CNPJ_DV"].astype(str).apply(lambda x: x.zfill(2), meta=('CNPJ_DV', str)))

df_estab['CNPJ'] = cnpjs

In [5]:
# df_estab.insert(0, "CNPJ", cnpjs, True) # insere a coluna CNPJ na primeira posição - não funciona pra 
df_estab['Pais'].unique().compute()

0      999
1      445
2      158
3      137
4      249
      ... 
191    464
192    165
193    833
194    824
195    494
Name: Pais, Length: 196, dtype: object

In [8]:
df_estab.head(5)

,CNPJ_Basico,CNPJ_Ordem,CNPJ_DV,Identificador,Nome_Fantasia,Situacao_Cadastral,Data_Situacao_Cadastral,Motivo_Situacao_Cadastral,Nome_Cidade_Exterior,Pais,...,DDD_1,Telefone_1,DDD_2,Telefone_2,DDD_Fax,Fax,Correio_Eletronico,Situacao_Especial,Data_Situacao_Especial,CNPJ
0,25084884,0001,30,1,INSTEC,08,20180427,01,NaN,999,...,38,98099427,NaN,NaN,NaN,NaN,wellervinicius@hotmail.com,NaN,NaN,25084884000130
1,25084895,0001,10,1,RANCHO DO ESPETINHO,08,20170720,01,NaN,999,...,27,97629715,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25084895000110
2,25084907,0001,07,1,FATIMA COSTURA,02,20160628,00,NaN,999,...,44,97179615,NaN,NaN,NaN,NaN,FERREIRADOSANTOSSILVAFATIMA@GMAIL.COM,NaN,NaN,25084907000107
3,00892104,0003,10,2,NaN,02,20160628,00,NaN,999,...,45,32241100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00892104000310
4,25084929,0001,77,1,NaN,04,20210225,63,NaN,999,...,51,98243624,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25084929000177


In [6]:
# df_estab[df_estab.duplicated(["CNPJ_Basico"], keep=False)].CNPJ_Basico.unique()
# df[df.duplicated(["Nome_Fantasia"], keep=False)].Nome_Fantasia.unique() #retorna os valores unicos que são duplicados na coluna informada
df_estab.isnull().sum().compute()

CNPJ_Basico                         0
CNPJ_Ordem                          0
CNPJ_DV                             0
Identificador                       0
Nome_Fantasia                18965090
Situacao_Cadastral                  0
Data_Situacao_Cadastral             0
Motivo_Situacao_Cadastral           0
Nome_Cidade_Exterior         45837310
Pais                                0
Data_Inicio_Atividad                0
CNAE_Fiscal_Principal               0
CNAE_Fiscal_Secundaria       26446794
Tipo_Logradouro                889492
Logradouro                       1991
Numero                           1660
Complemento                  26835630
Bairro                         369210
CEP                            138985
UF                                  1
Municipio                           1
DDD_1                        12186217
Telefone_1                   12186165
DDD_2                        42091058
Telefone_2                   42097566
DDD_Fax                      39366835
Fax         

In [ ]:
# df.loc[(df["Nome_Fantasia"] == "CIA DO SORRISO")] # retorna os registros onde o nome fantasia é igual ao valor informado
# df_estab.loc[(df_estab["CNPJ_Basico"] == 17069509)]

In [ ]:
df_estab.loc[(df_estab["Pais"] != 105)]

In [4]:
# df = spark.read.csv('data/extract/*.ESTABELE', sep = ';', inferSchema = True, header = True)
# df.createOrReplaceTempView spark.sql()